In [1]:
# Load data from CSV file into DataFrame
import pandas as pd

df = pd.read_csv('data/Retail sale of automotive fuel in specialised stores - Sheet1.csv', index_col=False)

# Replace spaces in the column names
df.columns = df.columns.str.strip()
df = df.T
df.columns = df.iloc[0]
df = df.drop(df.index[0])

# Cast the index to datetime (keeping only the year part)
df.index = pd.to_datetime(df.index, format='%Y')
df.index.name = 'Date'

# Ensure numeric data for interpolation (replace spaces and convert to numeric)
df = df.apply(lambda x: pd.to_numeric(x.str.replace(' ', '').str.strip(), errors='coerce'))

df["Original / Interpolated"] = True

# Resample and interpolate
df_resampled = df.resample('YS').mean()

interpolated = df_resampled[['Original / Interpolated']].copy()

df_resampled.drop(columns='Original / Interpolated', inplace=True)

interpolated.fillna('Interpolated', inplace=True)
interpolated.replace(True, 'Original', inplace=True)

df_interpolated = df_resampled.interpolate(method='linear', limit_direction='both', axis=0)

# Reset index to turn the datetime index into a column
data_for_plot = df_interpolated.reset_index()

# Extract the year from the datetime index
data_for_plot['Year'] = data_for_plot['Date'].dt.year
data_for_plot.set_index('Year', inplace=True)
data_for_plot.columns.name = None

data_for_plot = data_for_plot.drop(columns='Date')
data_for_plot['Sum'] = data_for_plot.sum(axis=1)

base_year = 2009
df_base = data_for_plot[data_for_plot.index == base_year]
data_for_plot_relative = data_for_plot.copy()

# Calculate relative values (set 2009 as 100%)
for col in df_base.columns:
    data_for_plot_relative[col] = (data_for_plot_relative[col] / df_base[col].values[0]) * 100

# Melt the original and relative DataFrames for plotting
# data_for_plot = data_for_plot.drop(columns=['Sum'])

pd.concat([data_for_plot.reset_index(drop=True), interpolated.reset_index(drop=True)], axis=1)

/var/folders/sc/xg8f6rss7r38560ppq_tr3680000gn/T/ipykernel_38659/89584949.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Interpolated' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  interpolated.fillna('Interpolated', inplace=True)


,Motor gasoline,"Auto diesel, dutiable","Auto diesel, free of duty",Sum,Original / Interpolated
0,1.620371e+06,1770911.0,88737.000000,3.480019e+06,Original
1,1.521132e+06,1904708.0,94107.000000,3.519947e+06,Original
2,1.374663e+06,2053294.0,96882.000000,3.524839e+06,Original
3,1.285134e+06,2173959.0,98947.000000,3.558040e+06,Original
4,1.203701e+06,2229014.0,94190.000000,3.526905e+06,Original
5,1.143917e+06,2296129.0,92260.000000,3.532306e+06,Original
6,1.080026e+06,2369138.0,94341.000000,3.543505e+06,Original
7,1.055713e+06,2442818.0,93396.000000,3.591927e+06,Original
8,1.014504e+06,2463081.0,86483.000000,3.564068e+06,Original
9,9.850780e+05,2381685.0,90447.000000,3.457210e+06,Original


In [2]:
pd.concat([data_for_plot_relative.reset_index(drop=True), interpolated.reset_index(drop=True)], axis=1)


,Motor gasoline,"Auto diesel, dutiable","Auto diesel, free of duty",Sum,Original / Interpolated
0,100.000000,100.000000,100.000000,100.000000,Original
1,93.875538,107.555264,106.051591,101.147350,Original
2,84.836312,115.945635,109.178809,101.287924,Original
3,79.311096,122.759359,111.505911,102.241971,Original
4,74.285519,125.868211,106.145125,101.347291,Original
5,70.595993,129.658069,103.970159,101.502492,Original
6,66.653007,133.780749,106.315291,101.824300,Original
7,65.152548,137.941319,105.250347,103.215730,Original
8,62.609365,139.085533,97.459910,102.415188,Original
9,60.793362,134.489254,101.927043,99.344573,Original


In [3]:
if 'Year' in data_for_plot.columns:
    data_for_plot = data_for_plot.set_index('Year')

# Apply heatmap styling using pandas built-in style
data_for_plot = data_for_plot.reset_index()

def format_number(x):
    return '{:,.0f}'.format(x).replace(',', ' ')


styled_df = (data_for_plot.style
             .background_gradient(cmap='Greens',subset=data_for_plot.columns[1:])
             .set_caption("")
             .hide(axis="index")
             .format(format_number, subset=data_for_plot.columns[1:]))
styled_df

Year,Motor gasoline,"Auto diesel, dutiable","Auto diesel, free of duty",Sum
2009,1 620 371,1 770 911,88 737,3 480 019
2010,1 521 132,1 904 708,94 107,3 519 947
2011,1 374 663,2 053 294,96 882,3 524 839
2012,1 285 134,2 173 959,98 947,3 558 040
2013,1 203 701,2 229 014,94 190,3 526 905
2014,1 143 917,2 296 129,92 260,3 532 306
2015,1 080 026,2 369 138,94 341,3 543 505
2016,1 055 713,2 442 818,93 396,3 591 927
2017,1 014 504,2 463 081,86 483,3 564 068
2018,985 078,2 381 685,90 447,3 457 210


In [4]:
from lets_plot import *
def main():
    LetsPlot.setup_html()
    global data_for_plot, data_for_plot_relative

    # Select only the columns you want to melt (excluding 'Sum' and 'index')
    columns_to_include = [col for col in data_for_plot.columns if col not in ['Sum', 'index']]
    
    # Melt the filtered DataFrame
    data_melted = pd.melt(data_for_plot[columns_to_include], id_vars='Year', var_name='Category', value_name='Value')
    data_relative_melted = pd.melt(data_for_plot_relative.reset_index(), id_vars='Year', var_name='Category', value_name='RelativeValue')
    
    # Create a wide plot for both actual and relative values
    plot = (ggplot(data_melted, aes(x='Year', y='Value', fill='Category'))
            + geom_area()
            + ggtitle('Stacked Values of All Categories (Excluding Timestamp)')
            + scale_x_continuous(breaks=list(data_for_plot.index))  # Ensure all years are displayed
            + labs(x='Year', y='Value')
            + ggsize(2000, 400))
    plot.show()

    # Dataset 1: All columns except 'Sum'
    data_without_sum = data_for_plot_relative[[col for col in data_for_plot_relative.columns if col != 'Sum']]
    data_relative_melted_without_sum = pd.melt(data_without_sum.reset_index(), id_vars='Year', var_name='Category', value_name='RelativeValue')

    # Dataset 2: Only the 'Sum' column
    data_sum_only = data_for_plot_relative[['Sum']]
    data_relative_melted_sum_only = pd.melt(data_sum_only.reset_index(), id_vars='Year', var_name='Category', value_name='RelativeValue')
    # Create a list of all unique years as strings for the x-axis breaks
    year_breaks = list(data_relative_melted_sum_only['Year'].unique())


    # Create a line plot for the relative values (with 2009 as 100%)
    plot_relative_all = (ggplot(data_relative_melted_without_sum, aes(x='Year', y='RelativeValue', color='Category', group='Category'))
                     + geom_line(size=1)  # Line chart
                     + geom_point(size=2)  # Optional points on the line
                     + ggtitle('Relative Values (2009 as 100%)')
                     + scale_x_continuous(breaks=year_breaks)  # Ensure all years are displayed
                     + labs(x='Year', y='Relative Value (%)')
                     + coord_cartesian(ylim=(0, None))  # Ensure y-axis starts from 0
                     + theme(axis_text_x=element_text(angle=90, hjust=1))  # Rotate x-axis labels for visibility
                     + ggsize(2000, 400))

    # Display the relative line plot
    plot_relative_all.show()



    # Create a line plot for the 'Sum' column
    plot_relative_sum = (ggplot(data_relative_melted_sum_only, aes(x='Year', y='RelativeValue', color='Category', group='Category'))
                        + geom_line(size=1)  # Line chart
                        + geom_point(size=2)  # Optional points on the line
                        + ggtitle('Relative Sum (2009 as 100%)')
                        + scale_x_continuous(breaks=year_breaks)  # Ensure all years are displayed
                        + labs(x='Year', y='Relative Value (%)')
                        + coord_cartesian(ylim=(0, None))  # Ensure y-axis starts from 0
                        + theme(axis_text_x=element_text(angle=90, hjust=1))  # Rotate x-axis labels for visibility
                        + ggsize(2000, 400))
    

# Display the relative line plot for the 'Sum' column
    plot_relative_sum.show()
main()
